# Forecasting Walmart's products prices

## Bartłomiej Adamiec

**Dlaczego te dane?**

Poszukiwałem datasetu, który będzie jak najbardziej komercyjny, abym mógł pokazać to w CV. Dataset o wyróżnia się na tle innych tym, że ma wpisane święta dla Stanów Zjednoczonych oraz dni, w których wydawane są kwoty pieniężne do spożytkowania na jedzenie dzięki programowi SNAP (czynniki egzogeniczne). Dataset wymaga prognozowania dla 30 490 równoległych szeregów czasowych zgrupowanych hierarchicznie Stany → Sklepy → Kategorie → Działy → Produkty.

**Cel:**

Celem analizy jest optymalizacja zarządzania zapasami w celu uniknięcia braków towarowych oraz nadmiernego magazynowania.


**Opis datasetu:**

Dane konkursowe (M5 Forecasting - Accuracy) z Kaggle. Zadaniem w konkursie było przewidzenie sprzedaży produktów w sieci sklepów Walmart na następne 28 dni. Dataset składa się z 5 plików csv przypominających relacyjną bazę danych w schemacie gwiazdy. Historia sprzedaży (`sales_train_validation.csv`) to główna tabela, a czas (`calendar.csv`) i cena (`sales_prices.csv`) to oddzielne tabele.

**Krótkie opisy plików:**

* `sell_prices.csv` - ceny produktów netto w USD
* `calendar.csv` - daty w różnej postaci oraz święta
* `sales_train_validation.csv` -  ile danego produktu sprzedało się w danym dniu. Dane w formacie szerokim. Dane obejmują zakres 1913 dni
* `sales_train_evaluation.csv` - identyczne do validation, tylko zawiera w sobie 28 następnych dni. Został dodany po zakończeniu konkursu.
* `sample_submission.csv` - przykład formatu w jakim uczestnicy powinni oddać zadanie

[Źródło](https://www.kaggle.com/competitions/m5-forecasting-accuracy/datam)

## Importing data

In [64]:
import pandas as pd
from pathlib import Path
data_directory = Path("m5-forecasting-accuracy")

data = {}

for file_path in data_directory.glob("*.csv"):
    file_name = file_path.stem

    print(f"Loading file: {file_name}")

    data[file_name] = pd.read_csv(file_path)

Loading file: sell_prices
Loading file: sample_submission
Loading file: calendar
Loading file: sales_train_validation
Loading file: sales_train_evaluation


## EDA

##### Sell prices

In [65]:
sell_prices = data['sell_prices']
sell_prices.head()
# sell_price is net price

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [70]:
print("Rows:", sell_prices.shape[0])
print("Stores:", sell_prices['store_id'].unique())
print("Sum of distinct items:", sell_prices['item_id'].nunique())

Rows: 6841121
Stores: ['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']
Sum of distinct items: 3049


##### Calendar

In [63]:
# wday = 1 is saturday!!!
calendar = data['calendar']
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [62]:
print("Rows:", calendar.shape[0])
print("Columns:", calendar.columns,"\n")
print("Event types:", calendar['event_type_1'].unique())
# SNAP - Supplemental Nutrition Assistance Program, if in row for snap_ column value is 1 that means in this day people in exact state have received money for food
# CA - California
# TX - Texas
# WI - Wisconsin

Rows: 1969
Columns: Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object') 

Event types: [nan 'Sporting' 'Cultural' 'National' 'Religious']


##### Validation

In [71]:
validation = data['sales_train_validation']
validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [73]:
# d_ is id for distinct day, value in column is quantity of product sold in this day

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1919)

##### Evaluation

In [74]:
evaluation = (data['sales_train_evaluation'])
evaluation.head()
# the same thing that is in validation but have 28 more days

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


#### Merge